## Proceso de Carga de Datos, Transformación y Almacenamiento
1. **Cargar datos de consumo mensual**: Cargar los archivos Excel (.xlsx) o Csv (.csv) en la carpeta *"data/consumo"*, con el formato de exportación normal. Una vez procesados estos se moverán a la carpeta *"data/procesados"*.
2. **Cargar datos de inspecciones**: Cargar los archivos Excel (.xlsx) o Csv (.csv) en la carpeta *"data/inspecciones"*. El formato esperado para estos archivos es el siguiente:  
    2.1 **Archivo de Inspecciones**: Debe contener el listado completo de inspecciones realizadas, tanto aquellas que resultaron en multa como las que no. Las columnas que debe contener este archivo son, mínimamente, las siguientes (los nombres deben coincidir de forma exacta):  
    > 2.1.1 *Cta.contrato*: Contiene el número de cuenta asociado a la inspección, que luego se vinculará con el número de instalación mediante el listado maestro de consumos.  
     > 2.1.2 *Fe.contab.*: Contiene la fecha de realización de la inspección al cliente. Su formato esperado es: **dd.mm.aaaa** (También debe coincidir de forma exacta el formato).

    2.2 **Archivo de multas**: Contendrá el listado de inspecciones que resultaron en aplicación de multas. Es importante mencionar que en el archivo de Inspecciones debe existir una contraparte para cada uno de los registros de multa, ya que para armar el dataset de entrenamiento es necesaria esta coincidencia completa. Las columnas que mínimamente deberá contener este archivo son (los nombres deben coincidir de forma exacta):
    > 2.2.1 *Instalación*: Número de instalación correspondiente al registro de multa (Número entero).  
    > 2.2.2 *Cuenta*: Número de cuenta asociado a la instalación en el registro de multa y en el registro maestro de consumos (Número entero).  
    > 2.2.3 *Fecha*: Fecha exacta de aplicación de la multa, en formato **dd/mm/aaaa**. Notar la diferencia de formato con la fecha de Inspección.
Una vez procesados estos archivos se moverán a la carpeta *"data/procesados"*.  
3. **Ejecutar celdas ETL**: Las celdas de abajo permiten obtener las series de consumo etiquetadas. Luego de ejecutarlas, se podrá llevar a cabo el entrenamiento del modelo de predicción en la notebook *train_predicciones.ipynb*, y con ello obtener las predicciones para guiar las inspecciones en campo.

In [2]:
######## CONFIGURACIONES PRELIMINARES ##########
## {Formato viejo de archivo de multas: **Recordar que la estructura de estos archivos debe contener una columna *"Código novedad"*, sin valor para las inspecciones sin multa y con algún valor para aquellas inspecciones con multa**.}
import os
import pandas as pd
import warnings
import io
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format #evita que muestre notacion cientifica
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
module_path = os.path.abspath(os.path.join('../../'))
import sys
if module_path not in sys.path:
    sys.path.append(module_path)
from src.preprocessing import etl

In [3]:
%%time
df_historico, df_instalacion_cuenta = etl.concatenar_nuevos_datos_consumo()

---Importación de datos nuevos de consumos---
Tamaño original df datos histórico: (16005327, 16)
Tamaño datos agregados: (0, 0)
Tamaño final df datos histórico: (16005327, 16)
Cantidad total de Cuentas: 703374
Cantidad total de Instalaciones: 683742
---Finalizado: Importación de datos nuevos de consumos---
CPU times: user 34.3 s, sys: 7.59 s, total: 41.9 s
Wall time: 1min 54s


In [5]:
#df_historico_inspecciones = etl.concatenar_nuevos_datos_inspecciones_formato_viejo()
df_historico_inspecciones = etl.concatenar_nuevos_datos_inspecciones()

---Importación de datos nuevos de inspecciones---
Tamaño original df datos histórico inspecciones: (10511, 9)
--- No se agregaron datos nuevos de inspecciones/multas


In [6]:
#df_fraudes_completo_etiquetado = etl.etiquetar_consumos_inspecciones_train_formato_viejo(df_historico,df_historico_inspecciones)
df_fraudes_completo_etiquetado = etl.etiquetar_consumos_inspecciones_train(df_historico,df_historico_inspecciones)

---Generación de datos de consumo etiquetados---
Tamaño original df datos histórico: (16005327, 16)
Tamaño original df datos histórico fraudes: (10511, 9)
Tamaño final df datos histórico completos etiquetados: (221533, 19)
---Finalizado: Generación de datos de consumo etiquetados---


In [7]:
df_wide_normal_and_fraud = etl.series_de_consumo_etiquetadas(df_fraudes_completo_etiquetado)

---Generación de series de consumo etiquetadas---


100%|██████████| 12/12 [00:14<00:00,  1.22s/it]

Tamaño final series etiquetadas: (4992, 32)
---Finalizado: Generación de series de consumo etiquetadas---


In [3]:
df_series_features = etl.cleaning_feature_engineering(df_wide_normal_and_fraud)

---Generación de features de series de consumo etiquetadas---
-- Usando datos históricos de feature eng: (4992, 115) ../../data/historico/df_series_features_train.parquet
Tamaño datos wide a agregar (nuevos datos no catalogados antes): (0, 32)
Tamaño final series con features: (4992, 115)
---Finalizado: Generación de features de series de consumo etiquetadas---


In [ ]:
%%time
X, y, df_features_selected = etl.feature_selection_dataset_entrenamiento(df_series_features)

In [7]:
%%time
series_mes = etl.series_de_consumo_mes_especifico(df_historico)#,mes='2022/12'

---Generación de series de consumo para predicción de mes específico: 2022/12---
Filtrado de datos del período
Consumos reales/estimados: Contabilizando cantidad de estimaciones
Estados: Contabilizando por tipo
Categorías: Contabilizando categorías del período y última categoría por instalación
Categorías: Identificando categorias mas frecuentes del período por instalación
Categorías: Contabilizando cambios
Iniciando armado de series
Iniciando unión de series y características
----Finalizado armado de series del mes----
CPU times: user 17min 32s, sys: 4.87 s, total: 17min 37s
Wall time: 17min 36s


In [11]:
%%time
series_features = cleaning_feature_engineering_mes_especifico(df_series_mes)

---Generación de features de series de consumo de mes específico---
*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***
Tamaño final series con features del mes: (1000, 112)
---Finalizado: Generación de features de series de consumo etiquetadas mes específico---
(1000, 112)


,instalacion,12_anterior,11_anterior,10_anterior,9_anterior,8_anterior,7_anterior,6_anterior,5_anterior,4_anterior,3_anterior,2_anterior,1_anterior,cant_consumo_est,cant_estado_0,cant_estado_1,cant_estado_2,cant_estado_3,cant_estado_4,mes,bimestre,trimestre,cuatrimestre,semestre,cant_categorias,ult_categoria,categ_mas_frecuente,cambios_categoria,index,0_Absolute energy,0_Average power,0_ECDF Percentile Count_0,0_ECDF Percentile Count_1,0_ECDF Percentile_0,0_ECDF Percentile_1,0_ECDF_0,0_ECDF_1,0_ECDF_2,0_ECDF_3,0_ECDF_4,0_ECDF_5,0_ECDF_6,0_ECDF_7,0_ECDF_8,0_ECDF_9,0_Entropy,0_Histogram_0,0_Histogram_1,0_Histogram_2,0_Histogram_3,0_Histogram_4,0_Histogram_5,0_Histogram_6,0_Histogram_7,0_Histogram_8,0_Histogram_9,0_Interquartile range,0_Kurtosis,0_Max,0_Mean,0_Mean absolute deviation,0_Median,0_Median absolute deviation,0_Min,0_Peak to peak distance,0_Root mean square,0_Skewness,0_Standard deviation,0_Variance,0_Area under the curve,0_Autocorrelation,0_Centroid,0_Mean absolute diff,0_Mean diff,0_Median absolute diff,0_Median diff,0_Negative turning points,0_Neighbourhood peaks,0_Positive turning points,0_Signal distance,0_Slope,0_Sum absolute diff,0_Zero crossing rate,mean_3,cant_ceros_3,max_cant_ceros_seg_3,slope_3,min_cons3,max_cons3,std_cons3,var_cons3,skew_cons3,mean_6,cant_ceros_6,max_cant_ceros_seg_6,slope_6,min_cons6,max_cons6,std_cons6,var_cons6,skew_cons6,kurt_cons6,mean_12,cant_ceros_12,max_cant_ceros_seg_12,slope_12,min_cons12,max_cons12,std_cons12,var_cons12,skew_cons12,kurt_cons12
0,50000000,664.00000,664.00000,527.00000,653.00000,131.00000,707.00000,784.00000,666.00000,681.00000,683.00000,649.00000,696.00000,1,3.00000,1.00000,4.00000,2.00000,1.00000,12,6,4,3,2,2,DOM,DOM,1,0,4997019.00000,45427445.45455,2.00000,9.00000,527.00000,683.00000,0.08333,0.16667,0.25000,0.33333,0.41667,0.50000,0.58333,0.66667,0.75000,0.83333,0.95351,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,34.25000,4.90408,784.00000,625.41667,98.80556,665.00000,17.00000,131.00000,653.00000,645.30477,-2.41337,158.97246,25272.24306,68.25000,4997019.00000,0.05821,150.36364,2.90909,77.00000,2.00000,4.00000,0.00000,3.00000,1655.31488,10.94755,1654.00000,0.00000,676.00000,0,0,6.50000,649.00000,696.00000,24.26932,589.00000,-1.18996,693.16667,0,0,-13.97143,649.00000,784.00000,47.32617,2239.76667,1.81247,3.86044,625.41667,0,0,10.94755,131.00000,784.00000,166.04132,27569.71970,-2.77275,8.52537
1,50000001,17.00000,21.00000,42.00000,18.00000,2.00000,27.00000,32.00000,41.00000,39.00000,32.00000,47.00000,31.00000,1,3.00000,3.00000,4.00000,1.00000,1.00000,12,6,4,3,2,3,SAL,DOM,3,1,11971.00000,108827.27273,2.00000,9.00000,17.00000,39.00000,0.08333,0.16667,0.25000,0.33333,0.41667,0.50000,0.58333,0.66667,0.75000,0.83333,0.95351,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,19.25000,-0.32329,47.00000,29.08333,10.06944,31.50000,10.00000,2.00000,45.00000,31.58454,-0.57502,12.31840,151.74306,3.25000,11971.00000,0.06729,13.09091,1.27273,15.00000,4.00000,2.00000,0.00000,3.00000,144.74499,1.90559,144.00000,0.00000,36.66667,0,0,-0.50000,31.00000,47.00000,8.96289,80.33333,1.70783,37.00000,0,0,0.17143,31.00000,47.00000,6.41872,41.20000,0.68746,-0.96404,29.08333,0,0,1.90559,2.00000,47.00000,12.86615,165.53788,-0.66065,0.21966
2,50000002,243.00000,227.00000,256.00000,240.00000,240.00000,258.00000,262.00000,214.00000,212.00000,221.00000,216.00000,215.00000,0,3.00000,1.00000,4.00000,2.00000,1.00000,12,6,4,3,2,3,NaN,DOM,7,2,658984.00000,5990763.63636,2.00000,9.00000,214.00000,243.00000,0.08333,0.16667,0.25000,0.33333,0.41667,0.50000,0.58333,0.66667,0.75000,0.83333,0.95351,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,30.50000,-1.43074,262.00000,233.66667,16.16667,233.50000,18.00000,212.00000,50.00000,234.34021,0.26041,17.75450,315.22222,25.75000,658984.00000,0.05198,13.45455,-2.54545,9.00000,-1.00000,2.00000,0.00000,3.00000,150.04564,-3.02797,148.00000,0.00000,217.33333,0,0,-3.00000,215

In [3]:
# REPORTE DE REGISTROS DE CONSUMO POR MES
df_historico.groupby('date').size()

date
2021-01-01    658163
2021-02-01    658562
2021-03-01    659023
2021-04-01    660299
2021-05-01    661579
2021-06-01    663180
2021-07-01    666178
2021-08-01    666749
2021-09-01    609019
2021-10-01    669822
2021-11-01    670203
2021-12-01    670790
2022-01-01    639999
2022-02-01    672971
2022-03-01    673833
2022-04-01    674394
2022-05-01    674872
2022-06-01    676186
2022-07-01    677301
2022-08-01    678540
2022-09-01    680011
2022-10-01    680776
2022-11-01    681040
2022-12-01    681837
dtype: int64